<a href="https://colab.research.google.com/github/AlexandreStackholders/AnhangueraADS/blob/main/Aula_1_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pacientes = []

def cadastrar_paciente():
    """Função para cadastrar um novo paciente."""
    print("\n--- CADASTRAR PACIENTE ---")
    nome = input("Nome do paciente: ")
    cpf = int(input("Cpf: "))
    idade = int(input("Idade: "))
    telefone = input("Telefone (DD) NNNNN-NNNN: ")

    paciente = {
        "nome": nome,
        "cpf": cpf,
        "idade": idade,
        "telefone": telefone
    }
    pacientes.append(paciente)
    print("Paciente cadastrado com sucesso!")

    # Ilustração visual do paciente sendo cadastrado.
    # Imagine uma ficha de paciente sendo preenchida.

def ver_estatisticas():
    """Função para exibir estatísticas da clínica."""
    print("\n--- ESTATÍSTICAS ---")
    total_pacientes = len(pacientes)
    print(f"Total de pacientes cadastrados: {total_pacientes}")

    if total_pacientes > 0:
        idades = [p['idade'] for p in pacientes]
        idade_media = sum(idades) / total_pacientes
        print(f"Idade média dos pacientes: {idade_media:.2f} anos")
    else:
        print("Nenhum paciente cadastrado para calcular estatísticas.")

    # Ilustração de um gráfico de pizza simples mostrando a distribuição de pacientes, talvez por idade.

def buscar_paciente():
    """Função para buscar um paciente pelo nome."""
    print("\n--- BUSCAR PACIENTE ---")
    termo_busca = input("Digite o nome do paciente para buscar: ").lower()

    encontrados = [p for p in pacientes if termo_busca in p['nome'].lower()]

    if encontrados:
        print("\nPacientes encontrados:")
        for paciente in encontrados:
            print(f"Nome: {paciente['nome']}, Cpf: {paciente['cpf']} , Idade: {paciente['idade']}, Telefone: {paciente['telefone']}")
    else:
        print("Nenhum paciente encontrado com esse nome.")

    # Ilustração de uma lupa procurando por nomes em uma lista.

def listar_todos_pacientes():
    """Função para listar todos os pacientes cadastrados."""
    print("\n--- LISTA DE TODOS OS PACIENTES ---")
    if not pacientes:
        print("Nenhum paciente cadastrado.")
    else:
        for i, paciente in enumerate(pacientes):
            print(f"[{i+1}] Nome: {paciente['nome']}, Cpf: {paciente['cpf']} , Idade: {paciente['idade']}, Telefone: {paciente['telefone']}")

    # Ilustração de uma lista de nomes em um caderno ou tela.

def menu_principal():
    """Função principal que exibe o menu e gerencia as opções."""
    while True:
        print("\n=== SISTEMA CLÍNICA VIDA+ ===")
        print("1. Cadastrar paciente")
        print("2. Ver estatísticas")
        print("3. Buscar paciente")
        print("4. Listar todos os pacientes")
        print("5. Sair")

        escolha = input("Escolha uma opção: ")

        if escolha == '1':
            cadastrar_paciente()
        elif escolha == '2':
            ver_estatisticas()
        elif escolha == '3':
            buscar_paciente()
        elif escolha == '4':
            listar_todos_pacientes()
        elif escolha == '5':
            print("Saindo do sistema. Até mais!")
            break
        else:
            print("Opção inválida. Por favor, escolha um número de 1 a 5.")

# Iniciar o aplicativo
if __name__ == "__main__":
    menu_principal()


=== SISTEMA CLÍNICA VIDA+ ===
1. Cadastrar paciente
2. Ver estatísticas
3. Buscar paciente
4. Listar todos os pacientes
5. Sair


KeyboardInterrupt: Interrupted by user

In [ ]:
import datetime
import sqlite3
import os

# Nome do arquivo do banco de dados SQLite
DB_FILE = 'clinica.db'

## ⚙️ FUNÇÕES DE ACESSO AO BANCO DE DADOS (DB)
#--------------------------------------------------------------------------------

def get_db_connection():
    """Cria e retorna uma conexão com o banco de dados."""
    # O objeto row_factory permite acessar colunas por nome (como dicionário)
    conn = sqlite3.connect(DB_FILE)
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    """Inicializa o banco de dados, criando a tabela de pacientes se não existir."""
    try:
        # Uso do 'with' garante que a conexão é fechada
        with get_db_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS pacientes (
                    id INTEGER PRIMARY KEY,
                    nome TEXT NOT NULL,
                    cpf TEXT UNIQUE NOT NULL,
                    idade INTEGER NOT NULL,
                    telefone TEXT
                )
            """)
            conn.commit()
        print(f"Banco de dados '{DB_FILE}' inicializado com sucesso.")
    except Exception as e:
        print(f"ERRO ao inicializar o banco de dados: {e}")

def get_all_patients_db():
    """Busca todos os pacientes no banco de dados."""
    with get_db_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM pacientes ORDER BY nome")
        pacientes = cursor.fetchall()
    return [dict(p) for p in pacientes]

def get_pacientes_para_exibir(termo_busca=None):
    """Busca pacientes (opcionalmente por termo) e os retorna como lista de dicionários."""
    with get_db_connection() as conn:
        cursor = conn.cursor()
        if termo_busca:
            cursor.execute("SELECT * FROM pacientes WHERE nome LIKE ? ORDER BY nome",
                           ('%' + termo_busca + '%',))
        else:
            cursor.execute("SELECT * FROM pacientes ORDER BY nome")
        encontrados = cursor.fetchall()
    return [dict(p) for p in encontrados]

#--------------------------------------------------------------------------------
## 🖥️ FUNÇÕES DE APRESENTAÇÃO E INTERAÇÃO
#--------------------------------------------------------------------------------

def print_box(title, content_lines, force_width=78):
    """
    Imprime um título e conteúdo dentro de uma caixa de linha dupla,
    usada APENAS para o menu principal.
    """
    box_width = force_width
    content_width = box_width - 4

    print("\n╔" + "═" * (box_width - 2) + "╗")

    # Centraliza o título
    print(f"║ {title.center(content_width)} ║")

    if content_lines:
        print("╠" + "═" * (box_width - 2) + "╣")
        # Imprime o conteúdo alinhado à esquerda
        for line in content_lines:
            # Garante que a linha caiba e fecha com a borda
            print(f"║ {line[:content_width].ljust(content_width)} ║")

    print("╚" + "═" * (box_width - 2) + "╝")

def validar_int_positivo(prompt, valor_atual=None):
    """Função para pedir e validar um número inteiro positivo."""
    while True:
        entrada = input(prompt)
        if not entrada and valor_atual is not None:
            return valor_atual

        if entrada.isdigit() and int(entrada) > 0:
            return int(entrada)

        print("Entrada inválida. Por favor, digite um número inteiro positivo ou deixe em branco para manter o atual.")

def selecionar_paciente(pacientes):
    """Função para selecionar um paciente pelo índice da lista exibida."""
    if not pacientes:
        print("Nenhum paciente disponível para seleção.")
        return None

    while True:
        try:
            escolha_indice = input("\nDigite o NÚMERO do paciente (o número entre [ ]): ")
            if not escolha_indice.isdigit():
                print("Entrada inválida. Por favor, digite apenas um número.")
                continue

            escolha_indice = int(escolha_indice) - 1
            if 0 <= escolha_indice < len(pacientes):
                return pacientes[escolha_indice]
            else:
                print("Número de paciente inválido.")
        except Exception:
            print("Entrada inválida.")

#--------------------------------------------------------------------------------
## ➕ FUNÇÕES DE MANIPULAÇÃO DE DADOS
#--------------------------------------------------------------------------------

def cadastrar_paciente():
    """Função para cadastrar um novo paciente no banco de dados."""
    print("\n--- CADASTRAR PACIENTE ---")
    try:
        nome = input("Nome do paciente: ")
        cpf = input("Cpf (somente números, ex: 12345678900): ")

        if not cpf.isdigit():
            print("ERRO: O CPF deve conter apenas números. Cadastro cancelado.")
            return

        idade = validar_int_positivo("Idade: ")
        if idade is None:
            print("Cadastro cancelado devido à idade inválida.")
            return

        telefone = input("Telefone (DD) NNNNN-NNNN: ")

        with get_db_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("INSERT INTO pacientes (nome, cpf, idade, telefone) VALUES (?, ?, ?, ?)",
                           (nome, cpf, idade, telefone))
            conn.commit()
        print("Paciente cadastrado com sucesso!")

    except sqlite3.IntegrityError:
        print("ERRO: Já existe um paciente cadastrado com este CPF.")
    except Exception as e:
        print(f"ERRO: Ocorreu um erro durante o cadastro: {e}. Cadastro cancelado.")

def listar_todos_pacientes():
    """Função para listar e exibir todos os pacientes cadastrados."""
    pacientes = get_pacientes_para_exibir()
    print("\n--- LISTA DE TODOS OS PACIENTES ---")
    if not pacientes:
        print("Nenhum paciente cadastrado.")
    else:
        for i, paciente in enumerate(pacientes):
            print(f"[{i+1}] (ID DB: {paciente['id']}) Nome: {paciente['nome']}, Cpf: {paciente['cpf']}, Idade: {paciente['idade']}, Telefone: {paciente['telefone']}")

    return pacientes

def ver_estatisticas():
    """Função para exibir estatísticas da clínica, buscando dados do DB."""
    pacientes = get_all_patients_db()
    total_pacientes = len(pacientes)

    print("\n--- ESTATÍSTICAS ---")
    print(f"Total de pacientes cadastrados: {total_pacientes}")

    if total_pacientes > 0:
        idades = [p['idade'] for p in pacientes]
        idade_media = sum(idades) / total_pacientes
        print(f"Idade média dos pacientes: {idade_media:.2f} anos")
    else:
        print("Nenhum paciente cadastrado para calcular estatísticas.")

def buscar_paciente():
    """Função para buscar um paciente pelo nome no banco de dados e exibir os resultados."""
    print("\n--- BUSCAR PACIENTE ---")
    termo_busca = input("Digite o nome do paciente para buscar: ")

    encontrados = get_pacientes_para_exibir(termo_busca)

    if encontrados:
        print(f"\nPacientes encontrados para '{termo_busca}':")
        for paciente in encontrados:
            print(f"(ID DB: {paciente['id']}) Nome: {paciente['nome']}, Cpf: {paciente['cpf']}, Idade: {paciente['idade']}, Telefone: {paciente['telefone']}")
    else:
        print(f"Nenhum paciente encontrado com o nome '{termo_busca}'.")

def editar_paciente():
    """Função para editar dados de um paciente existente no banco de dados."""
    print("\n--- EDITAR PACIENTE ---")
    pacientes_ativos = listar_todos_pacientes()
    if not pacientes_ativos:
        return

    paciente_escolhido = selecionar_paciente(pacientes_ativos)
    if not paciente_escolhido:
        return

    print(f"\nEditando: {paciente_escolhido['nome']} (ID DB: {paciente_escolhido['id']})")
    print("Deixe o campo em branco para manter o valor atual.")

    nome_atual = paciente_escolhido['nome']
    idade_atual = paciente_escolhido['idade']
    telefone_atual = paciente_escolhido['telefone']

    novo_nome = input(f"Novo Nome ({nome_atual}): ") or nome_atual

    nova_idade = validar_int_positivo(f"Nova Idade ({idade_atual}): ", idade_atual)
    if nova_idade is None:
        print("Edição cancelada devido à idade inválida.")
        return

    novo_telefone = input(f"Novo Telefone ({telefone_atual}): ") or telefone_atual

    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                UPDATE pacientes
                SET nome = ?, idade = ?, telefone = ?
                WHERE id = ?
            """, (novo_nome, nova_idade, novo_telefone, paciente_escolhido['id']))
            conn.commit()
        print(f"\nPaciente '{novo_nome}' editado com sucesso!")
    except Exception as e:
        print(f"ERRO ao editar paciente: {e}")

def excluir_paciente():
    """Função para excluir um paciente do banco de dados."""
    print("\n--- EXCLUIR PACIENTE ---")
    pacientes_ativos = listar_todos_pacientes()
    if not pacientes_ativos:
        return

    paciente_escolhido = selecionar_paciente(pacientes_ativos)
    if not paciente_escolhido:
        return

    confirmacao = input(f"Tem certeza que deseja EXCLUIR o paciente {paciente_escolhido['nome']} (S/N)? ").strip().upper()

    if confirmacao == 'S':
        try:
            with get_db_connection() as conn:
                cursor = conn.cursor()
                cursor.execute("DELETE FROM pacientes WHERE id = ?", (paciente_escolhido['id'],))
                conn.commit()
            print(f"\nPaciente '{paciente_escolhido['nome']}' excluído permanentemente.")
        except Exception as e:
            print(f"ERRO ao excluir paciente: {e}")
    else:
        print("Exclusão cancelada.")

def imprimir_cadastro():
    """Função para selecionar um paciente e 'imprimir' (salvar em arquivo TXT)."""
    print("\n--- IMPRIMIR/SALVAR CADASTRO (TXT) ---")
    pacientes_ativos = listar_todos_pacientes()
    if not pacientes_ativos:
        print("\nNenhum paciente cadastrado para imprimir.")
        return

    paciente_escolhido = selecionar_paciente(pacientes_ativos)
    if not paciente_escolhido:
        return

    paciente = paciente_escolhido
    data_hora_atual = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    conteudo_ficha = f"""
======================================
FICHA DE CADASTRO DO PACIENTE
======================================
Nome: {paciente['nome']}
CPF: {paciente['cpf']}
Idade: {paciente['idade']} anos
Telefone: {paciente['telefone']}
======================================
Gerado em: {data_hora_atual}
"""
    nome_arquivo = f"cadastro_{paciente['cpf']}.txt"

    try:
        with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
            arquivo.write(conteudo_ficha)

        print(f"\n--- IMPRESSÃO CONCLUÍDA ---")
        print(f"O cadastro de {paciente['nome']} foi salvo no arquivo: {nome_arquivo}")
        print("Você pode imprimir este arquivo manualmente.")

    except Exception as e:
        print(f"ERRO ao salvar o arquivo: {e}")

#--------------------------------------------------------------------------------
## 🏁 FUNÇÃO PRINCIPAL
#--------------------------------------------------------------------------------

def menu_principal():
    """Função principal que inicializa o DB, exibe o menu e gerencia as opções."""
    init_db()

    while True:
        menu_options = [
            "1. Cadastrar paciente",
            "2. Ver estatísticas",
            "3. Buscar paciente",
            "4. Listar todos os pacientes",
            "5. Editar paciente",
            "6. Excluir paciente",
            "7. Imprimir/Salvar Cadastro (TXT)",
            "",
            "8. Sair"
        ]
        # Único local onde a moldura de linhas duplas é usada
        print_box("SISTEMA ONG VIDA+", menu_options)

        escolha = input("Escolha uma opção: ")

        if escolha == '1':
            cadastrar_paciente()
        elif escolha == '2':
            ver_estatisticas()
        elif escolha == '3':
            buscar_paciente()
        elif escolha == '4':
            listar_todos_pacientes()
        elif escolha == '5':
            editar_paciente()
        elif escolha == '6':
            excluir_paciente()
        elif escolha == '7':
            imprimir_cadastro()
        elif escolha == '8':
            print("Saindo do sistema. Até mais!")
            break
        else:
            print("Opção inválida. Por favor, escolha um número de 1 a 8.")

if __name__ == "__main__":
    menu_principal()

Banco de dados 'clinica.db' inicializado com sucesso.

╔════════════════════════════════════════════════════════════════════════════╗
║                             SISTEMA ONG VIDA+                              ║
╠════════════════════════════════════════════════════════════════════════════╣
║ 1. Cadastrar paciente                                                      ║
║ 2. Ver estatísticas                                                        ║
║ 3. Buscar paciente                                                         ║
║ 4. Listar todos os pacientes                                               ║
║ 5. Editar paciente                                                         ║
║ 6. Excluir paciente                                                        ║
║ 7. Imprimir/Salvar Cadastro (TXT)                                          ║
║                                                                            ║
║ 8. Sair                                                                   